In [1]:
# Import useful libraries
import json, pandas as pd, pydeck as pdk, ee, ipyfilechooser, ipywidgets, datetime, sentinel_satellites

In [2]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Features Extraction

The project aim is to create a Machine Learning model capable of detecting the dates when a crop field has been manured, using satellite data.

Before even starting considering models or even doing an exploratory analysis, it is useful to extract all the features (both optical and radar) from sentinel satellites over the specified crop fields.

Feature extraction is fundamental in Earth Observation (EO) because it enables us to extract valuable information from large and complex satellite data. EO is a field that involves the collection and analysis of data about the Earth's surface and atmosphere from satellites and other airborne sensors. The data acquired from these sensors are usually vast, complex, and contain a wealth of information that needs to be extracted to make sense of them.

## Import a JSON file containing crop fields details

The structure of the JSON file must comply with two very simple rules ([example](../Datasets/main/main-fields.json)):
* it should contain the **crop field name** (preferred if each crop field have a unique name)
* it should contain the **set of coordinates** that are composing the single crop field (closed polygon geometry)

In [3]:
# Choose the file (it must be a JSON file)
file_chooser = ipyfilechooser.FileChooser(path='../../Datasets/main/', filename='main-fields.json', select_default=True, use_dir_icons=True, filter_pattern='*.json')
display(file_chooser)

FileChooser(path='/Users/francesco/Documents/Documents/Università (Università degli Studi, PV)/2_ MAGISTRALE…

In [4]:
# Load JSON data from file
with open(file_chooser.selected) as f:
    data = json.load(f)

# Create DataFrame with properties excluding 'manure_dates' column
fields_df = pd.DataFrame([{k:v for k,v in f['properties'].items() if k!='manure_dates'} for f in data['features']])

# Add column with coordinates for each field
fields_df['polygon_coordinates'] = [[tuple(c) for c in p] for f in data['features'] for p in f['geometry']['coordinates']]

# Create a dataframe that just has the columns crop_field_name and manure_dates
y_df = pd.DataFrame([{k:v for k,v in f['properties'].items() if k!='polygon_coordinates'} for f in data['features']])

In [5]:
# Show the entire dataframe
entire_df = fields_df.merge(y_df, on='crop_field_name')
entire_df

,crop_field_name,polygon_coordinates,manure_dates
0,P-BLD,"[(-4.202723286616649, 43.39683579015289), (-4....",[2022-05-26]
1,P-BLLT1,"[(-4.085622203603083, 43.429605845026266), (-4...",[2022-05-16]
2,P-BLLT2,"[(-4.084840437376829, 43.430826294936246), (-4...",[2022-05-26]
3,P-CBRCS1,"[(-4.200826431306206, 43.39067464298489), (-4....",[2022-05-26]
4,P-CBRCS2,"[(-4.204911872695676, 43.3876170244562), (-4.2...",[2022-05-26]
5,P-CLGT,"[(-4.111699726693341, 43.39830644556494), (-4....",[2022-05-16]
6,P-CLMBRS,"[(-4.544769098140127, 43.38040395682432), (-4....",[2022-05-26]
7,P-CMNTR,"[(-4.147208715069137, 43.40038457218137), (-4....",[2022-05-16]
8,P-DR,"[(-4.142486752802821, 43.396858931472195), (-4...",[2022-03-21]
9,P-FNFR,"[(-4.265940418729373, 43.38866671614796), (-4....",[2022-05-16]


## Show crop fields locations on Earth-map
The objective is to show where our crop fields are geographically placed. This can provide several benefits, including:
* **Spatial context:** it can provide a spatial context that makes it easier to understand the geographic distribution of the fields. This can be especially useful for people who are not familiar with the area or the crops being considered
* **Data exploration:** it can make it easier to explore the data contained in the JSON file. Users can zoom in and out, pan, and filter the data to focus on specific areas or types of crops
* **Data validation:** it can help validate the data contained in the JSON file. Users can visually confirm that the crop fields are located in the correct locations, and identify any potential errors or discrepancies in the data
* **Communication:** it can be a powerful way to communicate data to others. Users can share the map with stakeholders or the public to help them understand the geographic distribution of crop fields

In [6]:
# Define the layer with a tooltip
layer = pdk.Layer(
    'PolygonLayer',
    data=entire_df,
    get_polygon='polygon_coordinates',
    get_fill_color=[255, 255, 0, 100],
    get_line_color=[255, 255, 0, 100],
    stroked=True,
    filled=True,
    lineWidthMinPixels=3,
    pickable=True,
    auto_highlight=True,
)

# Define the initial view state of the map
view_state = pdk.ViewState(
    longitude=fields_df.polygon_coordinates[0][0][0],
    latitude=fields_df.polygon_coordinates[0][0][1],
    zoom=7.8
)

# Create the map with the layers and the initial view state
r = pdk.Deck(layers=layer, initial_view_state=view_state,)

# Show the map
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…

It can be noticed that our fields are placed in the Northern part of Spain. Please consider generalization issue.

## Features extraction - using sentinel-satellites PyPI library

The objective is to generate a dataset that contains for each field, for each time the satellites (sentinel 1 and sentinel 2) have passed on the field (in a period, specified by the user), all the phisical indicators that will be further used to build the final model.

This procedure has been designed to be performed in parallel in order to exploit the computational power of the machine (since each field is indipendent with the others).

Why you should use sentinel-satellites? [Sentinel-satellites](https://pypi.org/project/sentinel-satellites/) PyPI library provides **a useful, user friendly and powerful toolset for extracting Sentinel data in Python** (exploiting GEE APIs), and can be a valuable resource for researchers, analysts, and others working with Earth observation data.

### Select time-span to extract features

In [7]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=datetime.date(2022, 1, 1), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date(2022, 12, 31), disabled=False)
display(end_date_widget)

DatePicker(value=datetime.date(2022, 1, 1), description='Start date')

DatePicker(value=datetime.date(2022, 12, 31), description='End date')

### Sentinel 2 (optical features)

In [8]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 2 satellites
fields_s2_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=2, filters_params=['40'], fields_threads=4)
# Add manure dates
fields_s2_features_extracted_df = fields_s2_features_extracted_df.merge(y_df, on=str(y_df.columns[0]))

In [9]:
# Show the dataframe
fields_s2_features_extracted_df

,crop_field_name,s2_acquisition_date,B1,B2,B3,B4,B5,B6,B7,B8,...,CARI1,CARI2,MCARI,MCARI1,MCARI2,BSI,GLI,ALTERATION,SDI,manure_dates
0,P-BLD,2022-01-06,2.260204,119.981293,550.044218,234.045918,1055.875850,3447.054422,3945.947279,4264.421769,...,5303.671221,258.251928,3251.205159,6296.698571,0.922907,6282.955014,0.513069,2.131255,3484.508503,[2022-05-26]
1,P-BLD,2022-01-16,13.785714,111.540816,528.481293,224.115646,1062.081633,3571.093537,4166.239796,4458.365646,...,5524.406415,243.430799,3465.366528,6572.130408,0.921371,6545.200602,0.517949,2.127614,3636.382653,[2022-05-26]
2,P-BLD,2022-01-26,1092.221088,1174.479592,1585.828231,1284.115646,2188.639456,4718.545918,5334.006803,5582.107143,...,5638.426548,3807.506599,1336.179664,6659.779388,0.745694,10711.826348,0.126648,1.498058,3800.190476,[2022-05-26]
3,P-BLD,2022-01-31,3576.698980,3435.090136,3346.115646,3145.602041,3469.377551,4235.498299,4499.940476,4695.761905,...,7248.838290,6024.411146,329.911749,2545.031429,0.338758,14647.077956,0.008404,1.220073,1934.144558,[2022-05-26]
4,P-BLD,2022-02-05,1271.739796,1333.068027,1797.221088,1433.763605,2455.855442,5567.085034,6378.239796,6670.309524,...,6344.601486,4516.258365,1525.082405,8107.619796,0.759702,12721.888953,0.130101,1.549347,4551.319728,[2022-05-26]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,P-VNS,2022-10-08,6795.376682,6678.621525,6363.823318,6065.527354,6370.362332,6417.423318,6472.839462,6501.944395,...,13057.608911,8135.206560,318.781529,1093.782242,0.105144,24887.306273,-0.000648,1.071527,1238.345291,[2022-04-23]
851,P-VNS,2022-10-23,4059.667265,4026.379372,3709.677130,3366.669058,3713.954260,4235.782063,4364.606278,4536.184753,...,7809.331674,4330.322514,382.165375,2219.195193,0.305779,15406.636074,0.001776,1.091108,1350.292377,[2022-04-23]
852,P-VNS,2022-11-12,1443.568610,1602.507623,1970.007175,1972.767713,2587.695964,3854.926457,4186.630493,4406.135426,...,5741.673956,7573.688309,644.561034,3499.743067,0.496458,11958.219068,0.048533,1.385334,1536.214350,[2022-04-23]
853,P-VNS,2022-11-17,1133.057399,1364.994619,1726.755157,1631.587444,2309.713004,3881.580269,4315.133632,4503.217040,...,5345.539942,5596.065474,794.920534,4283.608251,0.594553,11195.020251,0.070841,1.441033,1939.695067,[2022-04-23]


#### Store the obtained dataset

In [10]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s2_features_extracted_df.to_csv(filename + '-s2-features-extracted.gz', header=True, index=False, compression='gzip')

### Sentinel 1 (radar features)

In [11]:
# Get all the mean features for the crop fields inside the dataframe, within a time period, using sentinel 1 satellites
fields_s1_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=1, filters_params=['ASCENDING'], fields_threads=4)
# Add manure dates
fields_s1_features_extracted_df = fields_s1_features_extracted_df.merge(y_df, on=str(y_df.columns[0]))

In [12]:
# Show the dataframe
fields_s1_features_extracted_df

,crop_field_name,s1_acquisition_date,VV,VH,AVE,DIF,RAT1,RAT2,NDI,RVI,manure_dates
0,P-BLD,2022-01-08,-13.310323,-20.305863,-16.808093,6.995540,0.655492,1.525573,-0.208100,2.416201,[2022-05-26]
1,P-BLD,2022-01-20,-14.468510,-19.414386,-16.941448,4.945876,0.745247,1.341837,-0.145970,2.291939,[2022-05-26]
2,P-BLD,2022-02-01,-13.837687,-20.370130,-17.103909,6.532442,0.679313,1.472076,-0.190963,2.381927,[2022-05-26]
3,P-BLD,2022-02-13,-14.072960,-20.416036,-17.244498,6.343076,0.689309,1.450728,-0.183916,2.367832,[2022-05-26]
4,P-BLD,2022-02-25,-14.952295,-20.977422,-17.964858,6.025127,0.712780,1.402957,-0.167692,2.335384,[2022-05-26]
...,...,...,...,...,...,...,...,...,...,...,...
958,P-VNS,2022-11-04,-10.377194,-16.707189,-13.542191,6.329995,0.621121,1.609991,-0.233714,2.467428,[2022-04-23]
959,P-VNS,2022-11-16,-10.902844,-17.969719,-14.436282,7.066875,0.606734,1.648168,-0.244761,2.489522,[2022-04-23]
960,P-VNS,2022-11-28,-9.501097,-16.193383,-12.847240,6.692286,0.586727,1.704370,-0.260456,2.520912,[2022-04-23]
961,P-VNS,2022-12-10,-10.948047,-18.870771,-14.909409,7.922724,0.580159,1.723666,-0.265695,2.531391,[2022-04-23]


#### Store the obtained dataset

In [13]:
# Compressed .csv files, to take less memory space
filename = file_chooser.selected_path + '/' + file_chooser.selected_filename.split('.')[0]
fields_s1_features_extracted_df.to_csv(filename + '-s1-features-extracted.gz', header=True, index=False, compression='gzip')